In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import os
import pandas as pd
from datetime import datetime, timedelta
import time
import json
from tqdm import tqdm
import re

In [2]:
headers = {
       'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebt/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36' 
    }
URL_PART_1 = 'https://api.reviews.2gis.com/2.0/branches/'
URL_PART_2 = '/reviews?fields=meta.providers%2Cmeta.branch_rating%2Cmeta.branch_reviews_count%2Cmeta.total_count%2Creviews.hiding_reason%2Creviews.is_verified&is_advertiser=false&key=37c04fe6-a560-4549-b459-02309cf643ad&limit=50&locale=ru_RU&offset_date='
URL_PART_3 = 'T12%3A27%3A50.824879%2B07%3A00&rated=true&sort_by=date_edited'

In [101]:
for city_file in os.listdir('2gis/review_hrefs'):
    city = city_file.split('.')[0]
    df = pd.DataFrame()
    with open(os.path.join('2gis/review_hrefs', city_file)) as file:
        urls_id = [url.strip() for url in file.readlines()]
    with requests.Session() as session:
#         ПО одному городу
        for url_id in tqdm(urls_id):
            date = str(datetime.now().date() + timedelta(days=1))
            offset = ''
            while True:
                url = URL_PART_1 + url_id + URL_PART_2 + date + URL_PART_3
                # print(url)
                response = session.get(url=url, headers=headers)
                js = response.json()['reviews']
                
                try:
                    # print(js)
                    date = js[-1]['date_created'][:10]
                    offset_now = js[0]['id']
                    if offset_now == offset:
                        break
                    offset = offset_now
                except IndexError:
                    break
                new = pd.DataFrame().from_dict(js)
                df = pd.concat([df,new])
                if len(js) == 1:
                    break
                
        df.to_csv(f'2gis\\company_info\\{city}.csv', index=False)
        

100%|██████████| 121/121 [00:08<00:00, 13.48it/s]


In [64]:
df = pd.DataFrame()

In [102]:
df = pd.DataFrame()
for file in os.listdir(f'2gis/company_info'):
    new = pd.read_csv(f'2gis/company_info/{file}')
    new['City'] = file.split('.')[0]
    # print(new)
    df = pd.concat([df, new])

In [103]:
df = df.loc[:, ['City', 'text', 'rating', 'date_created', 'user', 'object', 'official_answer']]

In [104]:
df = df.reset_index(drop=True)

In [105]:
df['date_created'] = df['date_created'].str[:10]
df['Comment author'] = df['user'].apply(lambda x: re.findall("'name': '(.+?)', 'p", x)[0])
df['Company_id'] = df['object'].apply(lambda x: re.findall("'id': '(.+?)', 'typ", x)[0])

In [106]:
df = df.drop(columns=['user', 'object', 'official_answer'])

In [108]:
df.to_csv('2gis_REVIEW_DATA.csv', index=False)

In [107]:
df

,City,text,rating,date_created,Comment author,Company_id
0,CHELYABINSK,УЖАСНЕЙШАЯ ШАРАШКИНА КОНТОРА!!! \nОбразовался ...,1.0,2021-09-28,Ekaterina Chaltceva,2111590606320294
1,CHELYABINSK,Совершенно некомпетентные сотрудники,1.0,2021-06-21,Ольга Красева,2111590606320294
2,CHELYABINSK,"Друзья, бегите от этих непрофессионалов!!! Раб...",1.0,2021-02-26,Вадим Сафин,2111590606320294
3,CHELYABINSK,"Недобропорядочная компания! Имейте ввиду, когд...",1.0,2021-02-25,Даша Иванова,2111590606320294
4,CHELYABINSK,Устанавливаю сигналку на квартиру. Менеджер Ни...,3.0,2020-11-22,Автор скрыл имя,2111590606320294
...,...,...,...,...,...,...
3573,VORONEZH,Ответственый Чоп. Качественое техническое обсл...,5.0,2021-02-25,Xoz Hunter,70000001041389608
3574,VORONEZH,"Была сработка в квартире, диспетчер позвонил и...",1.0,2021-10-26,bamemop514,4363391420281505
3575,YOSHKAR-OLA,Ваши сотрудники - ваше лицо. Сегодня 14.05.202...,1.0,2020-05-15,Анна Ильина,70000001028821608
3576,YOSHKAR-OLA,"аля 90-е , ничего не могут сказать или объясни...",1.0,2018-10-16,Vito Burberry,70000001023972860


In [2]:
df = pd.read_csv('2gis_REVIEW_DATA.csv')

In [4]:
df2 = pd.read_csv('2gis/ALL_DATA/BASE_INFO.csv')

In [6]:
df2 = df2.loc[:, ['Company', 'Company_id']]

In [7]:
df2

,Company,Company_id
0,Статус безопасности,70000001022001896
1,Арсенал,2111590606382024
2,ОМЕГА,70000001007323294
3,Кэндо,2111590606320587
4,Гридъ,70000001024594990
...,...,...
2812,Вымпел,9852152465858425
2813,Барс,9852152465851238
2814,Гранит-Безопасность,9852152465894303
2815,Русич,70000001050098511


In [11]:
df = df.merge(df2, how='left', on='Company_id')

In [13]:
df.to_csv('2gis_REVIEW_DATA.csv', index=False)